In [2]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 245.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 596.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 653.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 540.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 243.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 649.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 598.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 654.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 653.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nvidia-nvtx-cu12
    Found existing installation: nvidia-nvtx-cu12 12.4.99
    Uninstalling nvidia-nvtx-cu12-12.4.99:
      Successfully uninstalled nvidia-nvtx-cu12-12.4.99
  Attempting uninstall: nvidia-cusparse-cu12
  

In [3]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [4]:
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

test_ratio = 0.8

train_data = []
test_data = []

for type_name in set(dataset['type']):
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]

    test_size = int(len(curr_type_data) * test_ratio)

    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

def format_data(sample):
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])

    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

README.md:   0%|          | 0.00/909 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

원본 데이터의 type 분포:
synthetic_question: 497
paraphrased_question: 196
no_answer: 404
mrc_question_with_1_to_4_negative: 296
mrc_question: 491

전체 데이터 분할 결과: Train 380개, Test 1504개

학습 데이터의 type 분포:
synthetic_question: 100
paraphrased_question: 40
no_answer: 81
mrc_question_with_1_to_4_negative: 60
mrc_question: 99

테스트 데이터의 type 분포:
synthetic_question: 397
paraphrased_question: 156
no_answer: 323
mrc_question_with_1_to_4_negative: 236
mrc_question: 392


In [5]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: 폭스바겐에 이어 계열 브랜드인 아우디와 스코다도 디젤차 배출가스를 고의로 조작한 것으로 나타났다. 폭스바겐 그룹 산하에 있는 다른 브랜드로 ‘폭스바겐 스캔들’의 여파가 확산되는 모양새다.사건이 번지면서 세 가지 의혹이 꼬리를 물고 있다. 아우디는 210만대의 디젤차에 배출가스를 속이는 소프트웨어를 장착한 사실을 확인했다고 28일(현지시간) 발표했다. 이날 스코다도 아우디와 같은 눈속임 소프트웨어를 120만대의 디젤차에 적용했다고 밝혔다. 상용차 180만대와 그룹 내 나머지 5개 브랜드의 90만대 차량도 이번 사건에 연루된 것으로 폭스바겐그룹 측은 파악하고 있다.이번 사건을 폭스바겐 최고경영진이 사전에 알고 있었다는 의혹이 제기되고 있다. 마르틴 빈터코른 전 폭스바겐그룹 최고경영자(CEO)는 지난 23일 사퇴하면서 “부정행위에 놀랐지만 조작 사실을 전혀 몰랐다”고 부인했다. 하지만 27일 독일 일간지 FAZ는 “2011년 폭스바겐 내 기술자가 ‘배출가스 조작 행위가 이뤄지고 있다’고 보고했지만 묵살당했다”고 보도했다.이어 “8년간 폭스

In [6]:
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [7]:
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1504 [00:00<?, ? examples/s]

In [8]:
train_dataset[0]

{'messages': [{'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: 도시 농업의 경우, 세계 각지에서 지구 온난화 문제, 대기의 건조 및 미중 무역 분쟁, 한일 경제 전쟁 등의 여파에 따라 계속 발생되는 아마존 우림 산불 등과 같은 각종 자연재해 등이 지속적으로 일으키게 되면서, 기후 변화로 인한 생태계 파괴 등을 막을 수 있는 조건도 있고 자발적인 생활 패턴이 생기게 되는 대안을 제시할 수 있게 되는 이점이 있다. 그러나 도시 농업을 활성화하게 되면 숲과 같은 삼림 보호 정책은 물론 바다나 개펄 등 해양 생태계 보호 정책까지 대폭적으로 강화할 수 있는 등 다양한 니즈를 충족시킬 수 있게 된다. 기존에는 삼림 벌채 후 농경지 조성을 하였지만, 건물 최상층, 중간 피난층, 주상복합아파트 같은 주거 및 상업시설 복합 건축물의 상업 지역 부분 옥상 등지에 농지를 가꾸어져서 건물에 상주하고 있는 분들과 주거 시설 거주민들이 1년 동안 안정적으로 식량 자원을 확보할 수 있게 되는 좋은 점이 있다. 이미 EBS TV의 하나뿐인 지구라는 교양 프로그램에도 이와 같은 배경을 소재로 둔 이야기를 토대로 보면, 기후 변화 대응 체

In [9]:
model_id = "Qwen/Qwen2-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 도시 농업의 경우, 세계 각지에서 지구 온난화 문제, 대기의 건조 및 미중 무역 분쟁, 한일 경제 전쟁 등의 여파에 따라 계속 발생되는 아마존 우림 산불 등과 같은 각종 자연재해 등이 지속적으로 일으키게 되면서, 기후 변화로 인한 생태계 파괴 등을 막을 수 있는 조건도 있고 자발적인 생활 패턴이 생기게 되는 대안을 제시할 수 있게 되는 이점이 있다. 그러나 도시 농업을 활성화하게 되면 숲과 같은 삼림 보호 정책은 물론 바다나 개펄 등 해양 생태계 보호 정책까지 대폭적으로 강화할 수 있는 등 다양한 니즈를 충족시킬 수 있게 된다. 기존에는 삼림 벌채 후 농경지 조성을 하였지만, 건물 최상층, 중간 피난층, 주상복합아파트 같은 주거 및 상업시설 복합 건축물의 상업 지역 부분 옥상 등지에 농지를 가꾸어져서 건물에 상주하고 있는 분들과 주거 시설 거주민들이 1년 동안 안정적으로 식량 자원을 확보할 수 있게 되는 좋은 점이 있다. 이미 EBS TV의 하나뿐인 지구라는 교양 프로그램에도 이와 같은 배경을 소재로 둔 이야기를 토대로 보면, 기후 변화 대응 체계 해소 취지를 앞장서는 여건이 마련

In [11]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [12]:
args = SFTConfig(
    output_dir="qwen2-7b-rag-ko",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

In [13]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        labels = [-100] * len(input_ids)

        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    current_pos = assistant_pos + len(assistant_tokens)

                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):

                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

In [14]:
max_seq_length=8192

example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 4328])
어텐션 마스크 형태: torch.Size([1, 4328])
레이블 형태: torch.Size([1, 4328])


In [15]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 138520, 19391, 143604, 129264, 130650, 382, 13146, 48431, 20401, 66790, 29326, 131193, 17877, 125686, 125548, 139713, 624, 16, 13, 138520, 53680, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 143604, 16186, 139713, 624, 17, 13, 85322, 77226, 98801, 19391, 130768, 130213, 17877, 143604, 16186, 125476, 34395, 53900, 21329, 95577, 139713, 624, 18, 13, 138520, 19391, 128605, 143603, 12802, 85322, 77226, 98801, 19391, 130671, 32290, 85322, 77226, 98801, 126377, 330, 33883, 64795, 138520, 93, 19391, 128605, 130213, 12802, 136673, 1189, 5140, 45881, 34395, 143604, 16186, 139713, 624, 19, 13, 143604, 47836, 53618, 142976, 139236, 18411, 142616, 82190, 53435, 40853, 129549, 53435, 125068, 17877, 140174, 128836, 32290, 5140, 240, 97, 19391, 36330, 250, 125746, 16560, 23084, 126402, 83634, 17380, 94613, 139236, 84621, 47324, 18411, 129624, 20487, 139713, 13, 95617, 18411, 129901, 26698

In [16]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [17]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [18]:
trainer.train()

trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.568200
20,0.566200
30,0.426400
40,0.453700
50,0.407500
60,0.419500
70,0.384400
80,0.459700
90,0.455200
100,0.379700


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-pac

In [19]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [20]:
print(prompt_lst[42])

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: ‘관피아 논란’에도 불구하고 일찌감치 민간으로 옮겨 전문성과 능력을 인정받는 관료 출신들도 적지 않다. 대표적인 인물이 권용원 키움증권 사장이다. 권 사장은 1999년까지 산업자원부 산업기술개발과장(기술고시 21회)을 지내다가 이듬해 다우그룹 전략경영실장으로 이직했다. 다우그룹에서 신사업을 전담한 그는 신생 온라인 증권사였던 키움증권을 대형 증권사로 키워냈다.금융투자업계에선 전병조 KB투자증권 부사장이 눈에 띈다. 6년 전 기획재정부 관료(행정고시 29회)에서 기업금융(IB) 부문 전문가로 변신했다. 재정경제부 출신 이형승 케이스템셀 회장(행시 29회)은 삼성증권(이사) CJ그룹(소장) IBK투자증권(사장)을 거쳐 바이오회사로 갔다. 재정경제부 출신 이현승 전 SK증권 사장(행시 32회)도 AT커니(이사) 메릴린치(이사) GE에너지(사장) 등을 거쳤다.대기업 쪽에도 많다. 박영춘 SK 전무(행시 31회)는 금융위원회 금융정책과장을 거쳐 청와대 비상경제상황실 금융팀장을 지낸 금융 전문가로 SK그룹의 전략업무를 맡고 있다. 박 전무와 행시 동기인 문홍성 두산 부사장은 기획재정부 출신으

In [21]:
print(label_lst[42])


정부와 기업 간 협업의 사례와 그 결과는 다음과 같습니다.

1. **(주)이주와 정부의 협업**:
   - **사례**: 대구지역 섬유업체인 (주)이주는 2006년 정부로부터 부품소재전문기업으로 인정받고, 2008년에는 기술혁신형 중소기업으로 선정되었습니다. 이 회사는 정부 과제를 수주한 뒤 대구·경북 지역의 전문 연구소와 제휴를 맺어 연구 과제를 수행했습니다.
   - **결과**: 이러한 협업을 통해 (주)이주는 좋은 조건으로 자금을 사용할 수 있게 되었고, 대구시장 표창과 교육과학기술부 장관 표창을 받는 등 여러 성과를 거두었습니다. 이는 섬유업이 지식산업으로 탈바꿈할 수 있다는 가능성을 보여주는 사례로 평가받고 있습니다[[ref2]].

2. **정주영 현대그룹 명예회장과 박정희 대통령의 협업**:
   - **사례**: 1960년대 초 현대건설이 시공하는 소양강댐 건설 방식을 두고 정주영 회장과 박정희 대통령이 협업했습니다. 정 회장은 일본의 댐 설계 전문가와의 설전에서 소양강 주변의 자원을 이용한 사력댐 건설을 주장했고, 박 대통령은 정 회장의 손을 들어주었습니다.
   - **결과**: 이 협업의 결과로 공사비를 30% 절감할 수 있었고, 더 안전한 댐을 건설할 수 있었습니다[[ref5]].

3. **코오롱 창업주 이원만 회장과 박정희 대통령의 협업**:
   - **사례**: 이원만 회장은 1963년 박정희 대통령을 설득해 서울 구로동 수출공업단지 건설을 이끌어냈습니다. 그는 한국 근로자들의 손재주를 활용해 스테인리스 제품을 수출할 수 있다는 점을 강조했습니다.
   - **결과**: 이 협업을 통해 구로동 수출공업단지가 건설되었고, 이는 한국의 수출 산업 발전에 기여했습니다[[ref5]].

이와 같은 사례들은 정부와 기업 간의 협업이 기업의 성장과 국가 경제 발전에 중요한 역할을 할 수 있음을 보여줍니다.<|im_end|>



In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

In [3]:
model_id = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [5]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [6]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [7]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

NameError: name 'prompt_lst' is not defined

In [ ]:
base_model_id = "Qwen/Qwen2-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)